In [21]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [2]:
spark = SparkSession.builder.appName('EndSems4').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/26 21:50:31 WARN Utils: Your hostname, Karthikeya, resolves to a loopback address: 127.0.1.1; using 172.25.191.235 instead (on interface zt4homnczt)
25/10/26 21:50:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/26 21:50:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv('file:///home/karthikeya/Desktop/sem5/MIT_SEM5_BDA/Week4/datasets/movie_reviews.csv', header=True, inferSchema=True)

In [7]:
df = df.drop('review')

In [8]:
df.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    92|    176|   4.6|
|    30|     57|   2.5|
|    62|     64|   2.2|
|    64|    173|   2.1|
|    49|     44|   3.5|
+------+-------+------+
only showing top 5 rows


In [10]:
train, test = df.randomSplit([0.8, 0.2])

In [22]:
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', rank=10, regParam=0.2, maxIter=10, seed=819, coldStartStrategy='drop', nonnegative=True)

In [12]:
model = als.fit(train)

In [13]:
pred = model.transform(train)

In [16]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='mse')
mse = evaluator.evaluate(pred)

In [17]:
print("MSE: ", mse)

MSE:  0.13735402988042467


In [20]:
param_gris = ParamGridBuilder().addGrid(ALS.regParam, [0.2, 0.3]).addGrid(ALS.rank, [10, 20]).build()

In [23]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_gris, evaluator=evaluator)

In [24]:
models = cv.fit(train)
model = models.bestModel

In [25]:
spark.stop()